In [4]:
"""
A very simple and basic web scraping script. Feel free to
use this as a source of inspiration, but, make sure to attribute
it if you do so.

This is by no means production code.
"""
# built-in imports
import re
from json import dump
from collections import defaultdict

# user packages
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
import csv
import os

OUTPUT_DIR = "../data/raw/rent/"
headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}

In [7]:
def write_file(output_dir, file_name, content, mod="w"):
    """
    write given content to local file at: {output_dir}{file_name} with mode: {mod}
    output_dir: the String output folder directory, automatically create if not exist
    file_name: the String file name used to save file
    content: expecting objects which can be written to file with open function
    mod: String of writing mode code used in writing file
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    try:
        with open(f"{output_dir}{file_name}", mod) as f:
            f.write(content)
    except Exception as e:
        print(f"****** Writing file failure: {output_dir}{file_name}")
        print(e)
        print("******")
        return False
    return True



Retrieve post code

In [22]:
### Pull SA2 match table
url = "https://www.matthewproctor.com/Content/\
postcodes/australian_postcodes.csv"
response = requests.get(url, headers=headers)
write_file(f"{OUTPUT_DIR}", "post.csv", response.text)

# Select victoria data then save
vic_df = pd.read_csv(f"{OUTPUT_DIR}post.csv")
post_codes = vic_df.loc[vic_df["state"]=="VIC"]["postcode"].tolist()
post_codes = list(dict.fromkeys(post_codes))
print(len(post_codes))
print(post_codes[:10])


747
[3000, 3001, 3002, 3003, 3004, 3005, 3006, 3008, 3010, 3011]


Retrieve rent data

In [35]:
# /rent/melbourne-region-vic/?sort=price-desc&page=
# https://www.domain.com.au/rent/?postcode=3053&sort=default-desc

# constants
BASE_URL = "https://www.domain.com.au"

# begin code
url_links = []
max_page = 50
code_count = 0
file_size = 75
total_code = len(post_codes)
# generate list of urls to visit
while(code_count < len(post_codes)):
    post_code = post_codes[code_count]
    page_count = 0
    
    try:
        for page in range(1, max_page):
            url = BASE_URL + f"/rent/?postcode={post_code}&sort=default-desc&page={page}"
            #bs_object = BeautifulSoup(urlopen(url), "lxml")
            bs_object = BeautifulSoup(requests.get(url, headers=headers).text, "html.parser")
            # find the unordered list (ul) elements which are the results, then
            # find all href (a) tags that are from the base_url website.
            index_links = bs_object \
                .find(
                    "ul",
                    {"data-testid": "results"}
                ) \
                .findAll(
                    "a",
                    href=re.compile(f"{BASE_URL}/*") # the `*` denotes wildcard any
                )
            for link in index_links:
                # if its a property address, add it to the list
                if 'address' in link['class']:
                    url_links.append(link['href'])
            page_count += 1
    except Exception as e:
        print(f"{code_count}: {page_count} pages in post code: {post_code}")

        code_count += 1
        # output to example json in data/raw/
        if code_count%file_size == 0:
            print(f"File writed: {OUTPUT_DIR}url_part_{code_count//file_size}.csv")
            with open(f"{OUTPUT_DIR}url_part_{code_count//file_size}.csv", "w") as f:
                write = csv.writer(f)
                write.writerow(["url"])
                write.writerows([[i] for i in url_links])
            url_links = []
        
        continue
    
    code_count += 1
    # output to example json in data/raw/
    if code_count%file_size == 0:
        print(f"File writed: {OUTPUT_DIR}url_part_{code_count//file_size}.csv")
        with open(f"{OUTPUT_DIR}url_part_{code_count//file_size}.csv", "w") as f:
            write = csv.writer(f)
            write.writerow(["url"])
            write.writerows([[i] for i in url_links])
        url_links = []
        

# save urls
if code_count%file_size != 0:
    print(f"File writed: {OUTPUT_DIR}url_part_{code_count//file_size}.csv")
    with open(f"{OUTPUT_DIR}url_part_{code_count//file_size}.csv", "w") as f:
        write = csv.writer(f)
        write.writerow(["url"])
        write.writerows([[i] for i in url_links])
    url_links = []


https://www.domain.com.au/rent/?postcode=3084&sort=default-desc&page=1
https://www.domain.com.au/rent/?postcode=3084&sort=default-desc&page=2
https://www.domain.com.au/rent/?postcode=3084&sort=default-desc&page=3
https://www.domain.com.au/rent/?postcode=3084&sort=default-desc&page=4
76: 3 pages in post code: 3084
https://www.domain.com.au/rent/?postcode=3085&sort=default-desc&page=1
https://www.domain.com.au/rent/?postcode=3085&sort=default-desc&page=2
https://www.domain.com.au/rent/?postcode=3085&sort=default-desc&page=3
https://www.domain.com.au/rent/?postcode=3085&sort=default-desc&page=4
77: 3 pages in post code: 3085
https://www.domain.com.au/rent/?postcode=3086&sort=default-desc&page=1
78: 0 pages in post code: 3086
https://www.domain.com.au/rent/?postcode=3087&sort=default-desc&page=1
https://www.domain.com.au/rent/?postcode=3087&sort=default-desc&page=2
79: 1 pages in post code: 3087
https://www.domain.com.au/rent/?postcode=3088&sort=default-desc&page=1
https://www.domain.com.a

In [5]:
# read urls
with open(f"{OUTPUT_DIR}urls.csv", newline='\n') as f:
    reader = csv.reader(f)
    url_links = list(reader)
url_links = [i[0] for i in url_links][1:]

# for each url, scrape some basic metadata
part = 0
file_size = 50
count = part*file_size
property_metadata = defaultdict(dict)
for property_url in url_links[part*file_size:len(url_links)]:
    print(count, property_url)
    try:
        #bs_object = BeautifulSoup(urlopen(property_url), "lxml")
        bs_object = BeautifulSoup(requests.get(property_url, headers=headers).text, "html.parser")

        # looks for the header class to get property name
        property_metadata[property_url]['name'] = bs_object \
            .find("h1", {"class": "css-164r41r"}) \
            .text

        # looks for the div containing a summary title for cost
        property_metadata[property_url]['cost_text'] = bs_object \
            .find("div", {"data-testid": "listing-details__summary-title"}) \
            .text
        # extract coordinates from the hyperlink provided
        # i'll let you figure out what this does :P
        property_metadata[property_url]['coordinates'] = [
            float(coord) for coord in re.findall(
                r'destination=([-\s,\d\.]+)', # use regex101.com here if you need to
                bs_object \
                    .find(
                        "a",
                        {"target": "_blank", 'rel': "noopener noreferer"}
                    ) \
                    .attrs['href']
            )[0].split(',')
        ]
        property_metadata[property_url]['rooms'] = [
            re.findall(r'\d\s[A-Za-z]+', feature.text)[0] for feature in bs_object \
                .find("div", {"data-testid": "property-features"}) \
                .findAll("span", {"data-testid": "property-features-text-container"})
        ]
        property_metadata[property_url]['desc'] = re \
            .sub(r'<br\/>', '\n', str(bs_object.find("p"))) \
            .strip('</p >')
        count+=1
    except:
        count+=1
        if count%file_size == 0:
            print(f"File writed: ../data/raw/rent/rent_raw_part_{count//file_size}.json")
            with open(f'../data/raw/rent/rent_raw_part_{count//file_size}.json', 'w') as f:
                dump(property_metadata, f)
            property_metadata = defaultdict(dict)
        continue
    
    # output to example json in data/raw/
    if count%file_size == 0:
        print(f"File writed: ../data/raw/rent/rent_raw_part_{count//file_size}.json")
        with open(f'../data/raw/rent/rent_raw_part_{count//file_size}.json', 'w') as f:
            dump(property_metadata, f)
        property_metadata = defaultdict(dict)

    if count%file_size != 0:
    print(f"File writed: ../data/raw/rent/rent_raw_part_{count//file_size}.json")
    with open(f'../data/raw/rent/rent_raw_part_{count//file_size}.json', 'w') as f:
        dump(property_metadata, f)
    property_metadata = defaultdict(dict)


0 https://www.domain.com.au/12-macfarlan-street-south-yarra-vic-3141-16044182?topspot=1
1 https://www.domain.com.au/2-22-carr-street-brighton-east-vic-3187-16083096
2 https://www.domain.com.au/902-627-victoria-street-abbotsford-vic-3067-16083095
3 https://www.domain.com.au/133-ramsden-street-clifton-hill-vic-3068-16083078
4 https://www.domain.com.au/3-47-wellington-street-geelong-west-vic-3218-16082980
5 https://www.domain.com.au/3-113-osborne-street-south-yarra-vic-3141-16082959
6 https://www.domain.com.au/514-6-acacia-place-abbotsford-vic-3067-16082921
7 https://www.domain.com.au/4-165-clarke-street-northcote-vic-3070-16082860
8 https://www.domain.com.au/112a-bluff-road-black-rock-vic-3193-16082856
9 https://www.domain.com.au/47-gilbert-road-preston-vic-3072-16082846
10 https://www.domain.com.au/10-10-larbert-road-noble-park-vic-3174-16082810
11 https://www.domain.com.au/410-92-maroondah-highway-ringwood-vic-3134-16082788
12 https://www.domain.com.au/390-duke-street-sunshine-north-vi

In [3]:
property_df = pd.read_json('../data/raw/example.json')
print(property_df.head())

            https://www.domain.com.au/202-4-chapel-mews-south-yarra-vic-3141-16065573  \
name                    202/4 Chapel Mews South Yarra VIC 3141                          
cost_text                                        $600 per week                          
coordinates                  [-37.83562879999999, 144.9971116]                          
rooms                             [2 Beds, 2 Baths, 1 Parking]                          
desc                                              Georgia Lees                          

            https://www.domain.com.au/667-glenhuntly-road-caulfield-vic-3162-11598047  \
name                    667 Glenhuntly Road Caulfield VIC 3162                          
cost_text                   $38,000 p.a. Incl. Outgoings + GST                          
coordinates                         [-37.8860233, 145.0173065]                          
rooms                                                       []                          
desc                